# Day 3: Memory Systems & Knowledge Management

**Duration**: 2 hours  
**Learning Format**: 30 min theory + 60 min code + 30 min exercises  
**LLM Provider**: OpenAI (GPT-4, GPT-3.5-turbo)

## 📋 Learning Objectives
By the end of this session, you will:
- Understand different types of memory systems in AI agents
- Implement semantic, episodic, and procedural memory
- Integrate vector storage with OpenAI embeddings
- Build memory management tools for persistent knowledge
- Create agents with cross-session memory capabilities

## 📚 Prerequisites
- Completed Day 1 (LangGraph Foundations)
- Completed Day 2 (State Management & Persistence)
- OpenAI API key configured
- Basic understanding of embeddings and vector databases

## 🎯 What We'll Build Today
- Personal assistant with comprehensive memory
- Knowledge extraction and storage system
- Multi-domain memory management agent
- Vector-based semantic search capabilities

---
# 📖 Part 1: Learning Materials (30 minutes)

## 🎥 Video Resources

**Required Watching** (15 minutes):
- [Memory Systems in AI Agents](https://www.youtube.com/watch?v=memory-systems-example) - Overview of memory types
- [LangGraph Memory Management](https://www.youtube.com/watch?v=langgraph-memory-example) - Implementation patterns

**Optional Deep Dive** (30 minutes):
- [Vector Databases for AI](https://www.youtube.com/watch?v=vector-db-example) - Vector storage fundamentals
- [OpenAI Embeddings Guide](https://www.youtube.com/watch?v=openai-embeddings-example) - Best practices

## 📚 Documentation References
- [LangGraph Memory Documentation](https://langchain-ai.github.io/langgraph/concepts/memory/)
- [OpenAI Embeddings API](https://platform.openai.com/docs/guides/embeddings)
- [Vector Storage Integration](https://python.langchain.com/docs/integrations/vectorstores/)

## 🧠 Theory: Memory Systems in AI Agents

### Types of Memory Systems

#### 1. **Semantic Memory** 🧩
- **What**: General knowledge and facts
- **Examples**: "Paris is the capital of France", "Python is a programming language"
- **Implementation**: Vector storage with embeddings for similarity search
- **Use Cases**: Knowledge bases, fact retrieval, general information

#### 2. **Episodic Memory** 📖
- **What**: Specific events and experiences
- **Examples**: "User asked about weather on Tuesday", "Previous conversation about project X"
- **Implementation**: Timestamped conversation logs with context
- **Use Cases**: Conversation continuity, user preferences, interaction history

#### 3. **Procedural Memory** ⚙️
- **What**: Skills and procedures (how to do things)
- **Examples**: "How to process a refund", "Steps to deploy code"
- **Implementation**: Structured workflows and action sequences
- **Use Cases**: Business processes, automated workflows, skill execution

### Memory Management Strategies

#### **Short-term vs Long-term Memory**
```
Short-term (Session):
├── Current conversation context
├── Temporary variables
└── Active task state

Long-term (Persistent):
├── User preferences
├── Historical interactions
├── Learned knowledge
└── Accumulated experiences
```

#### **Namespace-based Organization**
- **User-specific**: `user:123:preferences`
- **Domain-specific**: `domain:finance:knowledge`
- **Temporal**: `session:2024-01-15:conversation`
- **Contextual**: `project:alpha:requirements`

## 🔧 Theory: Vector Storage Integration

### Why Vector Storage for Memory?
1. **Semantic Similarity**: Find related information based on meaning
2. **Scalability**: Handle large amounts of knowledge efficiently
3. **Flexibility**: Store and retrieve complex, unstructured information
4. **Context Awareness**: Understand relationships between concepts

### OpenAI Embeddings Strategy
```python
# Memory Storage Flow
Information → Embedding → Vector DB → Similarity Search → Retrieval
```

### Memory Lifecycle
1. **Encoding**: Convert information to embeddings
2. **Storage**: Save to vector database with metadata
3. **Retrieval**: Query based on similarity
4. **Integration**: Inject relevant memories into agent context
5. **Update**: Modify or expand existing memories

---
# 💻 Part 2: Hands-on Code Implementation (60 minutes)

## 🚀 Setup and Dependencies

In [ ]:
# Install required packages
!pip install langgraph openai pydantic chromadb python-dotenv sqlalchemy pandas numpy

In [ ]:
import os
import json
import sqlite3
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, TypedDict, Annotated
from dataclasses import dataclass
import uuid

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolExecutor

# OpenAI and Pydantic
import openai
from openai import OpenAI
from pydantic import BaseModel, Field

# Vector storage
import chromadb
from chromadb.config import Settings
import numpy as np

# Environment setup
from dotenv import load_dotenv
load_dotenv()

# Configure OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✅ All dependencies loaded successfully!")
print(f"📊 OpenAI API configured: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")

## 🧠 Section 1: Memory System Foundation

Let's start by building the core memory system architecture.

In [ ]:
# Memory System Data Models

class MemoryType:
    SEMANTIC = "semantic"      # Facts and knowledge
    EPISODIC = "episodic"      # Events and experiences
    PROCEDURAL = "procedural"  # Skills and procedures

class MemoryEntry(BaseModel):
    """Base memory entry model"""
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    content: str
    memory_type: str
    namespace: str
    timestamp: datetime = Field(default_factory=datetime.now)
    metadata: Dict[str, Any] = Field(default_factory=dict)
    embedding: Optional[List[float]] = None
    importance_score: float = Field(default=1.0, ge=0.0, le=1.0)

class SemanticMemory(MemoryEntry):
    """Semantic memory for facts and knowledge"""
    memory_type: str = Field(default=MemoryType.SEMANTIC)
    topic: str
    confidence: float = Field(default=1.0, ge=0.0, le=1.0)
    source: Optional[str] = None

class EpisodicMemory(MemoryEntry):
    """Episodic memory for events and experiences"""
    memory_type: str = Field(default=MemoryType.EPISODIC)
    event_type: str
    participants: List[str] = Field(default_factory=list)
    context: Dict[str, Any] = Field(default_factory=dict)
    duration: Optional[timedelta] = None

class ProceduralMemory(MemoryEntry):
    """Procedural memory for skills and procedures"""
    memory_type: str = Field(default=MemoryType.PROCEDURAL)
    skill_name: str
    steps: List[str]
    prerequisites: List[str] = Field(default_factory=list)
    success_rate: float = Field(default=1.0, ge=0.0, le=1.0)

# Agent State with Memory
class MemoryAgentState(TypedDict):
    messages: List[str]
    current_task: Optional[str]
    user_id: str
    session_id: str
    active_memories: List[MemoryEntry]
    memory_context: str
    last_retrieval: Optional[datetime]

print("✅ Memory system data models defined!")

## 🗄️ Section 2: Vector Storage with OpenAI Embeddings

Now let's implement vector storage using ChromaDB and OpenAI embeddings.

In [ ]:
class VectorMemoryStore:
    """Vector-based memory storage using ChromaDB and OpenAI embeddings"""
    
    def __init__(self, persist_directory: str = "./memory_db"):
        self.client = chromadb.PersistentClient(
            path=persist_directory,
            settings=Settings(allow_reset=True)
        )
        self.openai_client = OpenAI()
        
        # Create collections for different memory types
        self.semantic_collection = self._get_or_create_collection("semantic_memory")
        self.episodic_collection = self._get_or_create_collection("episodic_memory")
        self.procedural_collection = self._get_or_create_collection("procedural_memory")
        
    def _get_or_create_collection(self, name: str):
        """Get or create a ChromaDB collection"""
        try:
            return self.client.get_collection(name)
        except:
            return self.client.create_collection(name)
    
    async def get_embedding(self, text: str) -> List[float]:
        """Get OpenAI embedding for text"""
        try:
            response = self.openai_client.embeddings.create(
                input=text,
                model="text-embedding-3-small"  # Cost-effective option
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"❌ Error getting embedding: {e}")
            return []
    
    async def store_memory(self, memory: MemoryEntry) -> bool:
        """Store a memory entry with embedding"""
        try:
            # Get embedding for the content
            embedding = await self.get_embedding(memory.content)
            if not embedding:
                return False
            
            # Select appropriate collection
            collection = self._get_collection_for_type(memory.memory_type)
            
            # Prepare metadata
            metadata = {
                "namespace": memory.namespace,
                "timestamp": memory.timestamp.isoformat(),
                "importance_score": memory.importance_score,
                **memory.metadata
            }
            
            # Store in vector database
            collection.add(
                ids=[memory.id],
                embeddings=[embedding],
                documents=[memory.content],
                metadatas=[metadata]
            )
            
            print(f"✅ Stored {memory.memory_type} memory: {memory.id}")
            return True
            
        except Exception as e:
            print(f"❌ Error storing memory: {e}")
            return False
    
    def _get_collection_for_type(self, memory_type: str):
        """Get the appropriate collection for memory type"""
        if memory_type == MemoryType.SEMANTIC:
            return self.semantic_collection
        elif memory_type == MemoryType.EPISODIC:
            return self.episodic_collection
        elif memory_type == MemoryType.PROCEDURAL:
            return self.procedural_collection
        else:
            return self.semantic_collection  # Default
    
    async def search_memories(
        self, 
        query: str, 
        memory_type: Optional[str] = None,
        namespace: Optional[str] = None,
        limit: int = 5,
        min_similarity: float = 0.7
    ) -> List[Dict[str, Any]]:
        """Search for relevant memories"""
        try:
            # Get query embedding
            query_embedding = await self.get_embedding(query)
            if not query_embedding:
                return []
            
            results = []
            
            # Search in specified memory type or all types
            collections_to_search = []
            if memory_type:
                collections_to_search.append((self._get_collection_for_type(memory_type), memory_type))
            else:
                collections_to_search = [
                    (self.semantic_collection, MemoryType.SEMANTIC),
                    (self.episodic_collection, MemoryType.EPISODIC),
                    (self.procedural_collection, MemoryType.PROCEDURAL)
                ]
            
            for collection, mem_type in collections_to_search:
                # Build where clause for namespace filtering
                where_clause = {}
                if namespace:
                    where_clause["namespace"] = namespace
                
                # Search the collection
                search_results = collection.query(
                    query_embeddings=[query_embedding],
                    n_results=limit,
                    where=where_clause if where_clause else None
                )
                
                # Process results
                if search_results['documents'] and search_results['documents'][0]:
                    for i, doc in enumerate(search_results['documents'][0]):
                        distance = search_results['distances'][0][i]
                        similarity = 1 - distance  # Convert distance to similarity
                        
                        if similarity >= min_similarity:
                            results.append({
                                'content': doc,
                                'memory_type': mem_type,
                                'similarity': similarity,
                                'metadata': search_results['metadatas'][0][i],
                                'id': search_results['ids'][0][i]
                            })
            
            # Sort by similarity and return top results
            results.sort(key=lambda x: x['similarity'], reverse=True)
            return results[:limit]
            
        except Exception as e:
            print(f"❌ Error searching memories: {e}")
            return []
    
    def get_memory_stats(self) -> Dict[str, int]:
        """Get statistics about stored memories"""
        try:
            return {
                "semantic": self.semantic_collection.count(),
                "episodic": self.episodic_collection.count(),
                "procedural": self.procedural_collection.count()
            }
        except Exception as e:
            print(f"❌ Error getting stats: {e}")
            return {"semantic": 0, "episodic": 0, "procedural": 0}

# Initialize the vector memory store
memory_store = VectorMemoryStore()
print("✅ Vector memory store initialized!")
print(f"📊 Current memory stats: {memory_store.get_memory_stats()}")

## 🧰 Section 3: Memory Management Tools

Let's create tools for managing different types of memories.

In [ ]:
class MemoryTools:
    """Tools for memory management and operations"""
    
    def __init__(self, memory_store: VectorMemoryStore):
        self.memory_store = memory_store
    
    async def store_fact(self, content: str, topic: str, namespace: str = "general", 
                        confidence: float = 1.0, source: str = None) -> bool:
        """Store a semantic memory (fact)"""
        memory = SemanticMemory(
            content=content,
            topic=topic,
            namespace=namespace,
            confidence=confidence,
            source=source,
            importance_score=confidence
        )
        return await self.memory_store.store_memory(memory)
    
    async def store_experience(self, content: str, event_type: str, 
                              participants: List[str] = None, 
                              namespace: str = "general",
                              context: Dict[str, Any] = None) -> bool:
        """Store an episodic memory (experience)"""
        memory = EpisodicMemory(
            content=content,
            event_type=event_type,
            participants=participants or [],
            namespace=namespace,
            context=context or {}
        )
        return await self.memory_store.store_memory(memory)
    
    async def store_procedure(self, content: str, skill_name: str, 
                             steps: List[str], namespace: str = "general",
                             prerequisites: List[str] = None,
                             success_rate: float = 1.0) -> bool:
        """Store a procedural memory (skill/procedure)"""
        memory = ProceduralMemory(
            content=content,
            skill_name=skill_name,
            steps=steps,
            namespace=namespace,
            prerequisites=prerequisites or [],
            success_rate=success_rate
        )
        return await self.memory_store.store_memory(memory)
    
    async def recall_knowledge(self, query: str, namespace: str = None) -> List[Dict]:
        """Recall semantic knowledge related to query"""
        return await self.memory_store.search_memories(
            query=query,
            memory_type=MemoryType.SEMANTIC,
            namespace=namespace,
            limit=3
        )
    
    async def recall_experiences(self, query: str, namespace: str = None) -> List[Dict]:
        """Recall episodic experiences related to query"""
        return await self.memory_store.search_memories(
            query=query,
            memory_type=MemoryType.EPISODIC,
            namespace=namespace,
            limit=3
        )
    
    async def recall_procedures(self, query: str, namespace: str = None) -> List[Dict]:
        """Recall procedural knowledge related to query"""
        return await self.memory_store.search_memories(
            query=query,
            memory_type=MemoryType.PROCEDURAL,
            namespace=namespace,
            limit=3
        )
    
    async def comprehensive_recall(self, query: str, namespace: str = None) -> Dict[str, List]:
        """Comprehensive memory recall across all types"""
        knowledge = await self.recall_knowledge(query, namespace)
        experiences = await self.recall_experiences(query, namespace)
        procedures = await self.recall_procedures(query, namespace)
        
        return {
            "knowledge": knowledge,
            "experiences": experiences,
            "procedures": procedures
        }
    
    def format_memory_context(self, memories: Dict[str, List]) -> str:
        """Format memory recall results for agent context"""
        context_parts = []
        
        if memories["knowledge"]:
            context_parts.append("**Relevant Knowledge:**")
            for mem in memories["knowledge"]:
                context_parts.append(f"- {mem['content']} (confidence: {mem['similarity']:.2f})")
        
        if memories["experiences"]:
            context_parts.append("\n**Past Experiences:**")
            for mem in memories["experiences"]:
                context_parts.append(f"- {mem['content']} (relevance: {mem['similarity']:.2f})")
        
        if memories["procedures"]:
            context_parts.append("\n**Relevant Procedures:**")
            for mem in memories["procedures"]:
                context_parts.append(f"- {mem['content']} (relevance: {mem['similarity']:.2f})")
        
        return "\n".join(context_parts) if context_parts else "No relevant memories found."

# Initialize memory tools
memory_tools = MemoryTools(memory_store)
print("✅ Memory management tools initialized!")

## 🤖 Section 4: Memory-Enhanced Agent Implementation

Now let's create an agent that uses our memory system.

In [ ]:
class MemoryAgent:
    """LangGraph agent with comprehensive memory capabilities"""
    
    def __init__(self, memory_tools: MemoryTools, model: str = "gpt-4o-mini"):
        self.memory_tools = memory_tools
        self.model = model
        self.client = OpenAI()
        
        # Create the graph
        self.graph = self._create_graph()
    
    def _create_graph(self) -> StateGraph:
        """Create the memory agent graph"""
        graph = StateGraph(MemoryAgentState)
        
        # Add nodes
        graph.add_node("memory_retrieval", self._memory_retrieval_node)
        graph.add_node("agent_response", self._agent_response_node)
        graph.add_node("memory_storage", self._memory_storage_node)
        
        # Add edges
        graph.set_entry_point("memory_retrieval")
        graph.add_edge("memory_retrieval", "agent_response")
        graph.add_edge("agent_response", "memory_storage")
        graph.add_edge("memory_storage", END)
        
        return graph.compile()
    
    async def _memory_retrieval_node(self, state: MemoryAgentState) -> MemoryAgentState:
        """Retrieve relevant memories for the current task"""
        try:
            if not state["messages"]:
                return state
            
            # Get the latest message as query
            query = state["messages"][-1]
            namespace = f"user:{state['user_id']}"
            
            # Comprehensive memory recall
            memories = await self.memory_tools.comprehensive_recall(query, namespace)
            
            # Format memory context
            memory_context = self.memory_tools.format_memory_context(memories)
            
            # Update state
            state["memory_context"] = memory_context
            state["last_retrieval"] = datetime.now()
            
            print(f"🧠 Retrieved memories for query: '{query[:50]}...'")
            print(f"📝 Memory context length: {len(memory_context)} characters")
            
            return state
            
        except Exception as e:
            print(f"❌ Error in memory retrieval: {e}")
            state["memory_context"] = "No memories retrieved due to error."
            return state
    
    async def _agent_response_node(self, state: MemoryAgentState) -> MemoryAgentState:
        """Generate agent response using retrieved memories"""
        try:
            # Prepare system prompt with memory context
            system_prompt = f"""
You are an intelligent assistant with access to persistent memory. Use the following memory context to inform your responses:

{state['memory_context']}

Instructions:
1. Reference relevant memories when answering
2. Be specific about what you remember
3. Ask clarifying questions if memory context is insufficient
4. Acknowledge when you're learning something new
5. Be helpful and conversational

Current user: {state['user_id']}
Session: {state['session_id']}
"""
            
            # Prepare conversation history
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": state["messages"][-1]}
            ]
            
            # Get response from OpenAI
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.7,
                max_tokens=1000
            )
            
            assistant_response = response.choices[0].message.content
            
            # Add response to messages
            state["messages"].append(assistant_response)
            
            print(f"🤖 Generated response: '{assistant_response[:100]}...'")
            
            return state
            
        except Exception as e:
            print(f"❌ Error in agent response: {e}")
            error_response = "I apologize, but I encountered an error while processing your request."
            state["messages"].append(error_response)
            return state
    
    async def _memory_storage_node(self, state: MemoryAgentState) -> MemoryAgentState:
        """Store new information in memory"""
        try:
            if len(state["messages"]) < 2:
                return state
            
            user_message = state["messages"][-2]
            assistant_response = state["messages"][-1]
            namespace = f"user:{state['user_id']}"
            
            # Store the interaction as episodic memory
            interaction_content = f"User: {user_message}\nAssistant: {assistant_response}"
            await self.memory_tools.store_experience(
                content=interaction_content,
                event_type="conversation",
                participants=[state["user_id"], "assistant"],
                namespace=namespace,
                context={
                    "session_id": state["session_id"],
                    "task": state.get("current_task", "general_conversation")
                }
            )
            
            # Extract and store any facts mentioned by the user
            await self._extract_and_store_facts(user_message, namespace)
            
            print(f"💾 Stored interaction in episodic memory")
            
            return state
            
        except Exception as e:
            print(f"❌ Error in memory storage: {e}")
            return state
    
    async def _extract_and_store_facts(self, user_message: str, namespace: str):
        """Extract and store factual information from user message"""
        try:
            # Use OpenAI to extract facts
            extraction_prompt = f"""
Extract any factual statements or personal information from this message that should be remembered:

Message: "{user_message}"

Return ONLY facts that are:
1. Factual statements (not opinions)
2. Personal information about the user
3. Preferences or settings
4. Important details to remember

Format as a simple list, one fact per line. If no facts, return "None".
"""
            
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",  # Use cheaper model for extraction
                messages=[{"role": "user", "content": extraction_prompt}],
                temperature=0.1,
                max_tokens=300
            )
            
            facts_text = response.choices[0].message.content.strip()
            
            if facts_text and facts_text.lower() != "none":
                facts = [f.strip() for f in facts_text.split('\n') if f.strip()]
                
                for fact in facts:
                    await self.memory_tools.store_fact(
                        content=fact,
                        topic="user_info",
                        namespace=namespace,
                        confidence=0.9,
                        source="user_conversation"
                    )
                
                print(f"📚 Extracted and stored {len(facts)} facts")
            
        except Exception as e:
            print(f"⚠️ Error extracting facts: {e}")
    
    async def chat(self, message: str, user_id: str, session_id: str = None) -> str:
        """Chat with the memory-enhanced agent"""
        if not session_id:
            session_id = str(uuid.uuid4())
        
        # Create initial state
        initial_state = MemoryAgentState(
            messages=[message],
            current_task=None,
            user_id=user_id,
            session_id=session_id,
            active_memories=[],
            memory_context="",
            last_retrieval=None
        )
        
        # Run the graph
        final_state = await self.graph.ainvoke(initial_state)
        
        # Return the assistant's response
        return final_state["messages"][-1]

# Initialize the memory agent
memory_agent = MemoryAgent(memory_tools)
print("✅ Memory-enhanced agent initialized!")

## 🧪 Section 5: Testing the Memory System

Let's test our memory system with some examples.

In [ ]:
# Test the memory system
async def test_memory_system():
    """Test the memory system functionality"""
    
    print("🧪 Testing Memory System...\n")
    
    # Test 1: Store some initial facts
    print("📚 Storing initial knowledge...")
    await memory_tools.store_fact(
        content="Python is a high-level programming language known for its readability",
        topic="programming",
        namespace="general",
        confidence=1.0
    )
    
    await memory_tools.store_fact(
        content="LangGraph is a library for building stateful multi-agent applications",
        topic="programming",
        namespace="general",
        confidence=0.95
    )
    
    await memory_tools.store_procedure(
        content="To deploy a Python application: 1) Test locally, 2) Build container, 3) Deploy to production",
        skill_name="python_deployment",
        steps=["Test locally", "Build container", "Deploy to production"],
        namespace="general"
    )
    
    print("✅ Initial knowledge stored!\n")
    
    # Test 2: Chat with the agent
    print("💬 Testing agent conversation...")
    
    # First conversation
    response1 = await memory_agent.chat(
        message="Hi! My name is Alice and I'm learning Python programming.",
        user_id="alice_123"
    )
    print(f"🤖 Agent: {response1}\n")
    
    # Second conversation - should remember Alice
    response2 = await memory_agent.chat(
        message="Can you tell me about LangGraph?",
        user_id="alice_123"
    )
    print(f"🤖 Agent: {response2}\n")
    
    # Third conversation - should remember context
    response3 = await memory_agent.chat(
        message="How do I deploy Python applications?",
        user_id="alice_123"
    )
    print(f"🤖 Agent: {response3}\n")
    
    # Test with different user
    response4 = await memory_agent.chat(
        message="What do you know about me?",
        user_id="bob_456"
    )
    print(f"🤖 Agent (Bob): {response4}\n")
    
    # Alice again - should remember previous conversations
    response5 = await memory_agent.chat(
        message="What did we discuss earlier?",
        user_id="alice_123"
    )
    print(f"🤖 Agent (Alice): {response5}\n")
    
    # Display memory statistics
    stats = memory_store.get_memory_stats()
    print(f"📊 Final Memory Statistics: {stats}")

# Run the test
await test_memory_system()

## 🔍 Section 6: Memory Query and Analysis Tools

Let's add some tools for analyzing and querying our memory system.

In [ ]:
class MemoryAnalyzer:
    """Tools for analyzing and querying memory contents"""
    
    def __init__(self, memory_store: VectorMemoryStore):
        self.memory_store = memory_store
    
    async def search_user_memories(self, user_id: str, query: str = "") -> Dict[str, List]:
        """Search all memories for a specific user"""
        namespace = f"user:{user_id}"
        
        if query:
            # Search with specific query
            results = await self.memory_store.search_memories(
                query=query,
                namespace=namespace,
                limit=10,
                min_similarity=0.5
            )
        else:
            # Get all memories for user (using a very general query)
            results = await self.memory_store.search_memories(
                query="conversation interaction user",
                namespace=namespace,
                limit=20,
                min_similarity=0.0
            )
        
        # Organize by memory type
        organized = {"semantic": [], "episodic": [], "procedural": []}
        for result in results:
            memory_type = result.get("memory_type", "semantic")
            if memory_type in organized:
                organized[memory_type].append(result)
        
        return organized
    
    def display_user_memory_summary(self, memories: Dict[str, List], user_id: str):
        """Display a formatted summary of user memories"""
        print(f"\n👤 Memory Summary for User: {user_id}")
        print("=" * 50)
        
        total_memories = sum(len(memories[key]) for key in memories)
        print(f"📊 Total Memories: {total_memories}")
        
        for memory_type, items in memories.items():
            if items:
                print(f"\n🧠 {memory_type.upper()} MEMORIES ({len(items)}):")
                print("-" * 30)
                
                for i, item in enumerate(items[:5], 1):  # Show top 5
                    content = item['content'][:100] + "..." if len(item['content']) > 100 else item['content']
                    timestamp = item.get('metadata', {}).get('timestamp', 'Unknown')
                    similarity = item.get('similarity', 0)
                    
                    print(f"{i}. {content}")
                    print(f"   📅 {timestamp} | 🔗 Relevance: {similarity:.2f}\n")
                
                if len(items) > 5:
                    print(f"   ... and {len(items) - 5} more memories\n")
    
    async def analyze_user_interests(self, user_id: str) -> List[str]:
        """Analyze user interests based on their memory patterns"""
        memories = await self.search_user_memories(user_id)
        
        # Extract topics and keywords
        all_content = []
        for memory_type in memories.values():
            for memory in memory_type:
                all_content.append(memory['content'])
        
        if not all_content:
            return []
        
        # Use OpenAI to analyze interests
        combined_content = "\n".join(all_content[:10])  # Limit content
        
        analysis_prompt = f"""
Analyze the following conversation history and memory content to identify the user's main interests and topics of discussion:

{combined_content}

Return a list of 3-5 main interests or topics, one per line. Be specific and concise.
"""
        
        try:
            client = OpenAI()
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": analysis_prompt}],
                temperature=0.3,
                max_tokens=200
            )
            
            interests = [line.strip() for line in response.choices[0].message.content.strip().split('\n') if line.strip()]
            return interests
            
        except Exception as e:
            print(f"❌ Error analyzing interests: {e}")
            return []
    
    async def memory_search_demo(self, query: str):
        """Demonstrate memory search capabilities"""
        print(f"\n🔍 Searching memories for: '{query}'")
        print("=" * 50)
        
        results = await self.memory_store.search_memories(
            query=query,
            limit=5,
            min_similarity=0.3
        )
        
        if results:
            for i, result in enumerate(results, 1):
                content = result['content'][:150] + "..." if len(result['content']) > 150 else result['content']
                print(f"{i}. [{result['memory_type'].upper()}] {content}")
                print(f"   🔗 Similarity: {result['similarity']:.3f} | 📅 {result.get('metadata', {}).get('timestamp', 'Unknown')}\n")
        else:
            print("No relevant memories found.")

# Initialize memory analyzer
memory_analyzer = MemoryAnalyzer(memory_store)
print("✅ Memory analyzer initialized!")

In [ ]:
# Demonstrate memory analysis capabilities
async def demo_memory_analysis():
    """Demonstrate memory analysis tools"""
    
    print("🔬 Memory Analysis Demo\n")
    
    # Analyze Alice's memories
    alice_memories = await memory_analyzer.search_user_memories("alice_123")
    memory_analyzer.display_user_memory_summary(alice_memories, "alice_123")
    
    # Analyze Alice's interests
    interests = await memory_analyzer.analyze_user_interests("alice_123")
    print(f"\n🎯 Alice's Interests: {interests}")
    
    # Demonstrate search capabilities
    await memory_analyzer.memory_search_demo("Python programming")
    await memory_analyzer.memory_search_demo("deployment")

# Run the demo
await demo_memory_analysis()

---
# 🎯 Part 3: Practical Exercises (30 minutes)

## Exercise 1: Personal Assistant with Memory

**Goal**: Build a personal assistant that remembers user preferences and past interactions.

**Your Task**: Complete the implementation below to create a sophisticated personal assistant.

In [ ]:
# Exercise 1: Personal Assistant with Memory

class PersonalAssistant:
    """Personal assistant with comprehensive memory"""
    
    def __init__(self, memory_agent: MemoryAgent):
        self.memory_agent = memory_agent
        self.memory_tools = memory_agent.memory_tools
    
    async def setup_user_profile(self, user_id: str, profile_data: Dict[str, Any]):
        """Set up initial user profile in memory"""
        namespace = f"user:{user_id}"
        
        # TODO: Store user profile information as semantic memories
        # Hint: Use memory_tools.store_fact() for each piece of profile data
        
        for key, value in profile_data.items():
            # YOUR CODE HERE
            await self.memory_tools.store_fact(
                content=f"User's {key}: {value}",
                topic="user_profile",
                namespace=namespace,
                confidence=1.0,
                source="user_setup"
            )
        
        print(f"✅ User profile set up for {user_id}")
    
    async def handle_request(self, request: str, user_id: str) -> str:
        """Handle user request with memory context"""
        # TODO: Use the memory agent to process the request
        # Hint: Use self.memory_agent.chat()
        
        response = await self.memory_agent.chat(request, user_id)
        return response
    
    async def learn_preference(self, user_id: str, preference: str, category: str):
        """Learn and store a user preference"""
        namespace = f"user:{user_id}"
        
        # TODO: Store the preference as a semantic memory
        # Hint: Include the category in the topic or metadata
        
        await self.memory_tools.store_fact(
            content=f"User prefers: {preference}",
            topic=f"preference_{category}",
            namespace=namespace,
            confidence=0.9,
            source="user_interaction"
        )
        
        print(f"📚 Learned preference: {preference} (category: {category})")

# Test the Personal Assistant
async def test_personal_assistant():
    print("🤖 Testing Personal Assistant...\n")
    
    assistant = PersonalAssistant(memory_agent)
    
    # Set up user profile
    profile = {
        "name": "Sarah",
        "job": "Data Scientist",
        "location": "San Francisco",
        "interests": "Machine Learning, Python, Hiking"
    }
    
    await assistant.setup_user_profile("sarah_789", profile)
    
    # Test conversations
    requests = [
        "What's my name and what do I do for work?",
        "I prefer coffee over tea",
        "Recommend some weekend activities",
        "What do you know about my preferences?"
    ]
    
    for request in requests:
        if "prefer coffee" in request:
            await assistant.learn_preference("sarah_789", "coffee over tea", "beverages")
        
        response = await assistant.handle_request(request, "sarah_789")
        print(f"👤 Sarah: {request}")
        print(f"🤖 Assistant: {response}\n")

# Run the test
await test_personal_assistant()

## Exercise 2: Knowledge Extraction System

**Goal**: Create a system that extracts and organizes knowledge from documents or conversations.

**Your Task**: Implement a knowledge extraction and organization system.

In [ ]:
# Exercise 2: Knowledge Extraction System

class KnowledgeExtractor:
    """System for extracting and organizing knowledge"""
    
    def __init__(self, memory_tools: MemoryTools):
        self.memory_tools = memory_tools
        self.client = OpenAI()
    
    async def extract_knowledge_from_text(self, text: str, domain: str = "general") -> Dict[str, List[str]]:
        """Extract different types of knowledge from text"""
        
        extraction_prompt = f"""
Analyze the following text and extract knowledge in these categories:

Text: "{text}"

Extract:
1. FACTS: Factual statements and information
2. PROCEDURES: Step-by-step processes or instructions
3. CONCEPTS: Important concepts or definitions

Format as:
FACTS:
- fact 1
- fact 2

PROCEDURES:
- procedure 1
- procedure 2

CONCEPTS:
- concept 1
- concept 2

If a category has no items, write "None".
"""
        
        try:
            # TODO: Use OpenAI to extract knowledge
            # Hint: Use self.client.chat.completions.create()
            
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": extraction_prompt}],
                temperature=0.1,
                max_tokens=800
            )
            
            # Parse the response
            extracted_text = response.choices[0].message.content
            knowledge = self._parse_extracted_knowledge(extracted_text)
            
            # Store the extracted knowledge
            await self._store_extracted_knowledge(knowledge, domain)
            
            return knowledge
            
        except Exception as e:
            print(f"❌ Error extracting knowledge: {e}")
            return {"facts": [], "procedures": [], "concepts": []}
    
    def _parse_extracted_knowledge(self, text: str) -> Dict[str, List[str]]:
        """Parse the extracted knowledge text"""
        knowledge = {"facts": [], "procedures": [], "concepts": []}
        
        # TODO: Parse the text to extract facts, procedures, and concepts
        # Hint: Split by sections and extract items
        
        sections = text.split('\n\n')
        current_section = None
        
        for line in text.split('\n'):
            line = line.strip()
            if line.startswith('FACTS:'):
                current_section = 'facts'
            elif line.startswith('PROCEDURES:'):
                current_section = 'procedures'
            elif line.startswith('CONCEPTS:'):
                current_section = 'concepts'
            elif line.startswith('- ') and current_section:
                item = line[2:].strip()
                if item.lower() != 'none':
                    knowledge[current_section].append(item)
        
        return knowledge
    
    async def _store_extracted_knowledge(self, knowledge: Dict[str, List[str]], domain: str):
        """Store extracted knowledge in memory"""
        namespace = f"domain:{domain}"
        
        # Store facts as semantic memories
        for fact in knowledge["facts"]:
            await self.memory_tools.store_fact(
                content=fact,
                topic=domain,
                namespace=namespace,
                confidence=0.8,
                source="knowledge_extraction"
            )
        
        # Store procedures as procedural memories
        for i, procedure in enumerate(knowledge["procedures"]):
            # TODO: Store procedures using memory_tools.store_procedure()
            # Hint: You might need to break down the procedure into steps
            
            steps = [step.strip() for step in procedure.split(',') if step.strip()]
            if not steps:
                steps = [procedure]
            
            await self.memory_tools.store_procedure(
                content=procedure,
                skill_name=f"{domain}_procedure_{i+1}",
                steps=steps,
                namespace=namespace
            )
        
        # Store concepts as semantic memories with concept topic
        for concept in knowledge["concepts"]:
            await self.memory_tools.store_fact(
                content=concept,
                topic=f"{domain}_concepts",
                namespace=namespace,
                confidence=0.9,
                source="knowledge_extraction"
            )
        
        print(f"📚 Stored: {len(knowledge['facts'])} facts, {len(knowledge['procedures'])} procedures, {len(knowledge['concepts'])} concepts")

# Test the Knowledge Extraction System
async def test_knowledge_extraction():
    print("🧠 Testing Knowledge Extraction System...\n")
    
    extractor = KnowledgeExtractor(memory_tools)
    
    # Sample text about machine learning
    sample_text = """
    Machine learning is a subset of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed. 
    
    To train a machine learning model, follow these steps: 1) Collect and clean your data, 2) Choose an appropriate algorithm, 3) Split data into training and testing sets, 4) Train the model on training data, 5) Evaluate performance on test data, 6) Fine-tune hyperparameters if needed.
    
    Key concepts include supervised learning (learning with labeled data), unsupervised learning (finding patterns in unlabeled data), and reinforcement learning (learning through rewards and penalties). Neural networks are computational models inspired by biological neural networks.
    """
    
    # Extract knowledge
    knowledge = await extractor.extract_knowledge_from_text(sample_text, "machine_learning")
    
    # Display results
    print("\n📊 Extracted Knowledge:")
    for category, items in knowledge.items():
        print(f"\n{category.upper()}:")
        for item in items:
            print(f"  - {item}")
    
    # Test retrieval
    print("\n🔍 Testing knowledge retrieval...")
    ml_knowledge = await memory_tools.recall_knowledge("machine learning training", "domain:machine_learning")
    for knowledge in ml_knowledge:
        print(f"  📝 {knowledge['content'][:100]}...")

# Run the test
await test_knowledge_extraction()

## 🏆 Challenge Exercise: Multi-Domain Memory Agent

**Goal**: Create an advanced agent that can manage memories across multiple domains and provide domain-specific expertise.

**Your Task**: Implement a sophisticated multi-domain memory system.

In [ ]:
# Challenge Exercise: Multi-Domain Memory Agent

class MultiDomainMemoryAgent:
    """Advanced agent with multi-domain memory management"""
    
    def __init__(self, memory_tools: MemoryTools):
        self.memory_tools = memory_tools
        self.client = OpenAI()
        self.domains = ["technology", "business", "science", "arts", "sports"]
    
    async def classify_domain(self, query: str) -> str:
        """Classify the domain of a query"""
        # TODO: Use OpenAI to classify the domain of the query
        # Hint: Create a prompt that asks GPT to classify the query into one of the domains
        
        classification_prompt = f"""
Classify the following query into one of these domains: {', '.join(self.domains)}

Query: "{query}"

Return only the domain name, nothing else.
"""
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": classification_prompt}],
                temperature=0.1,
                max_tokens=20
            )
            
            domain = response.choices[0].message.content.strip().lower()
            return domain if domain in self.domains else "technology"  # Default
            
        except Exception as e:
            print(f"❌ Error classifying domain: {e}")
            return "technology"  # Default domain
    
    async def provide_domain_expertise(self, query: str, user_id: str) -> str:
        """Provide domain-specific expertise"""
        # TODO: Implement domain-specific expertise
        # 1. Classify the domain
        # 2. Retrieve domain-specific memories
        # 3. Retrieve user-specific memories
        # 4. Generate a response using both contexts
        
        # Step 1: Classify domain
        domain = await self.classify_domain(query)
        print(f"🎯 Classified domain: {domain}")
        
        # Step 2: Retrieve domain-specific knowledge
        domain_namespace = f"domain:{domain}"
        domain_memories = await self.memory_tools.comprehensive_recall(query, domain_namespace)
        
        # Step 3: Retrieve user-specific memories
        user_namespace = f"user:{user_id}"
        user_memories = await self.memory_tools.comprehensive_recall(query, user_namespace)
        
        # Step 4: Generate response
        domain_context = self.memory_tools.format_memory_context(domain_memories)
        user_context = self.memory_tools.format_memory_context(user_memories)
        
        system_prompt = f"""
You are a domain expert in {domain}. Use the following contexts to provide a comprehensive answer:

DOMAIN EXPERTISE ({domain}):
{domain_context}

USER CONTEXT:
{user_context}

Provide a detailed, expert-level response that:
1. Leverages domain-specific knowledge
2. Considers the user's background and interests
3. Provides actionable insights
4. Asks relevant follow-up questions if appropriate
"""
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": query}
                ],
                temperature=0.7,
                max_tokens=1000
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            print(f"❌ Error generating response: {e}")
            return "I apologize, but I encountered an error while processing your request."
    
    async def add_domain_knowledge(self, domain: str, content: str, knowledge_type: str = "fact"):
        """Add knowledge to a specific domain"""
        namespace = f"domain:{domain}"
        
        if knowledge_type == "fact":
            await self.memory_tools.store_fact(
                content=content,
                topic=domain,
                namespace=namespace,
                confidence=0.9,
                source="domain_knowledge"
            )
        elif knowledge_type == "procedure":
            # Parse procedure into steps
            steps = [step.strip() for step in content.split('.') if step.strip()]
            await self.memory_tools.store_procedure(
                content=content,
                skill_name=f"{domain}_procedure",
                steps=steps,
                namespace=namespace
            )
        
        print(f"📚 Added {knowledge_type} to {domain} domain")

# Test the Multi-Domain Memory Agent
async def test_multi_domain_agent():
    print("🌐 Testing Multi-Domain Memory Agent...\n")
    
    agent = MultiDomainMemoryAgent(memory_tools)
    
    # Add some domain-specific knowledge
    await agent.add_domain_knowledge(
        "technology",
        "Cloud computing provides on-demand access to computing resources over the internet",
        "fact"
    )
    
    await agent.add_domain_knowledge(
        "business",
        "A successful startup requires: market research, product development, funding, team building, and customer acquisition",
        "procedure"
    )
    
    await agent.add_domain_knowledge(
        "science",
        "CRISPR-Cas9 is a revolutionary gene-editing technology that allows precise modifications to DNA",
        "fact"
    )
    
    # Test domain classification and expertise
    test_queries = [
        "How does cloud computing work and what are its benefits?",
        "What steps should I take to start a tech startup?",
        "Explain how CRISPR gene editing technology works",
        "What are the latest trends in artificial intelligence?"
    ]
    
    for query in test_queries:
        print(f"\n👤 User: {query}")
        response = await agent.provide_domain_expertise(query, "expert_user_001")
        print(f"🤖 Expert Agent: {response[:200]}...\n")

# Run the test
await test_multi_domain_agent()

---
# 🎯 Summary and Next Steps

## 📚 What You've Learned Today

### ✅ Key Concepts Mastered:
1. **Memory Types**: Semantic, episodic, and procedural memory systems
2. **Vector Storage**: Integration with OpenAI embeddings for semantic search
3. **Memory Management**: Tools for storing, retrieving, and organizing memories
4. **Cross-Session Persistence**: Maintaining memory across agent interactions
5. **Domain-Specific Knowledge**: Organizing memory by namespaces and domains

### 🛠️ Technical Skills Developed:
- ChromaDB vector storage implementation
- OpenAI embeddings integration
- Memory-enhanced LangGraph agents
- Knowledge extraction and organization
- Multi-domain memory management

### 🏗️ Projects Completed:
1. **Vector Memory Store**: Full implementation with ChromaDB and OpenAI
2. **Memory-Enhanced Agent**: LangGraph agent with comprehensive memory
3. **Personal Assistant**: Agent that remembers user preferences and history
4. **Knowledge Extractor**: System for extracting and organizing information
5. **Multi-Domain Agent**: Advanced agent with domain-specific expertise

## 🚀 Next Steps

### Tomorrow's Focus (Day 4):
- **Multi-Agent Communication**: Learn how agents communicate and coordinate
- **Handoff Mechanisms**: Implement agent-to-agent task handoffs
- **Command Objects**: Structured communication patterns
- **Supervisor Patterns**: Orchestrating multiple specialized agents
- **Security**: Secure data injection and validation

### 💡 Homework Suggestions:
1. Extend the personal assistant with more sophisticated preference learning
2. Implement memory aging and forgetting mechanisms
3. Add memory compression for long-term storage efficiency
4. Experiment with different embedding models and vector databases
5. Build a domain-specific knowledge base for your area of expertise

### 📖 Additional Resources:
- [Vector Database Comparison Guide](https://example.com/vector-db-guide)
- [OpenAI Embeddings Best Practices](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings)
- [Memory Systems in AI Research](https://example.com/memory-systems-research)
- [LangGraph Memory Documentation](https://langchain-ai.github.io/langgraph/concepts/memory/)

## 🎉 Congratulations!

You've successfully implemented a comprehensive memory system for AI agents! You now understand how to:
- Store and retrieve different types of memories
- Use vector databases for semantic search
- Build agents that learn and remember across sessions
- Extract and organize knowledge from text
- Manage domain-specific expertise

**Ready for Day 4?** Tomorrow we'll explore how multiple agents can work together, communicate effectively, and hand off tasks to create powerful multi-agent systems!